## Importing libraries and defining relevant folders

#### Importing libraries

In [1]:
import os, sys, time, datetime, glob, bcolz, random

In [2]:
import shutil

In [3]:
import keras
import numpy as np
import pandas as pd

/pylon5/ir5fpcp/amogh112/faus_dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
from keras import Sequential

In [5]:
import matplotlib.pyplot as plt

In [6]:
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Convolution2D, Input, BatchNormalization

In [7]:
from keras_vggface.vggface import VGGFace

#### Relevant folders

In [15]:
!ls $SCRATCH/AffectNet/data_copy

2  Manually_Annotated_Images


In [16]:
folder_affectnet = "/pylon5/ir5fpcp/amogh112/AffectNet/"

In [22]:
folder_train_data = folder_affectnet + 'data_copy/Manually_Annotated_Images/'

## Generating labels for valence and arousal training

#### Reading and filtering dataframes from csv files

In [18]:
path_training_csv = folder_affectnet + "Manually_Annotated_file_lists/training_va.csv"
folder_annotated_training_images = folder_affectnet + "Manually_Annotated_Images/"

Getting image_filename and the corresponding labels for valence and arousal

In [19]:
df_train = pd.read_csv(path_training_csv,header=None)
df_train.head()

,0,1,2
0,689/737db2483489148d783ef278f43f486c0a97e140fc...,0.785714,-0.055556
1,392/c4db2f9b7e4b422d14b6e038f0cdc3ecee239b5532...,-0.017253,0.004313
2,468/21772b68dc8c2a11678c8739eca33adb6ccc658600...,0.174603,0.007937
3,944/06e9ae8d3b240eb68fa60534783eacafce2def60a8...,0.153401,0.038890
4,993/02e06ee5521958b4042dd73abb444220609d96f57b...,0.783972,-0.551684


In [23]:
list_image_paths = df_train.iloc[:,0].values

In [24]:
df_va_train = df_train.iloc[:,[0,len(df_train.columns)-2,len(df_train.columns)-1]]
df_va_train.columns = ["image","valence", "arousal"]
print("shape is: ", df_va_train.shape)
(df_va_train.head())

shape is:  (414799, 3)


,image,valence,arousal
0,689/737db2483489148d783ef278f43f486c0a97e140fc...,0.785714,-0.055556
1,392/c4db2f9b7e4b422d14b6e038f0cdc3ecee239b5532...,-0.017253,0.004313
2,468/21772b68dc8c2a11678c8739eca33adb6ccc658600...,0.174603,0.007937
3,944/06e9ae8d3b240eb68fa60534783eacafce2def60a8...,0.153401,0.038890
4,993/02e06ee5521958b4042dd73abb444220609d96f57b...,0.783972,-0.551684


Converting the dataframe in a dictionary

In [25]:
dict_im_va = df_va_train.set_index('image').T.to_dict('list')
dict_im_va

{'689/737db2483489148d783ef278f43f486c0a97e140fc4b6b61b84363ca.jpg': [0.785714,
  -0.055555600000000004],
 '392/c4db2f9b7e4b422d14b6e038f0cdc3ecee239b55326e9181ee4520f9.jpg': [-0.0172533,
  0.00431332],
 '468/21772b68dc8c2a11678c8739eca33adb6ccc658600e4da2224080603.jpg': [0.174603,
  0.00793651],
 '944/06e9ae8d3b240eb68fa60534783eacafce2def60a86042f9b7d59544.jpg': [0.153401,
  0.0388903],
 '993/02e06ee5521958b4042dd73abb444220609d96f57b1689abbe87c024.jpg': [0.783972,
  -0.5516840000000001],
 '979/f675c6a88cdef99a6d8b0261741217a0319387fcf1571a174f99ac81.jpg': [-0.34126999999999996,
  0.547619],
 '637/94b769d8e880cbbea8eaa1350cb8c094a03d27f9fef44e1f4c0fb2ae.jpg': [-2.0,
  -2.0],
 '997/b81f843f08ce3bb0c48b270dc58d2ab8bf5bea3e2262e50bbcadbec2.jpg': [-0.488145,
  0.831839],
 '358/21a32dd1c1ecd57d3e8964621c911df1c0b3348a4ae5203b4a243230.JPG': [-2.0,
  -2.0],
 '330/60216cf5171a1f18876f178e4ab84db22f2dc0920bdaca819769b032.jpg': [-2.0,
  -2.0],
 '981/5c991a28b5550cf5e1e830d5eed1dc7f80581462d627

#### Moving the files with bad labels from the training set

In [26]:
image_name_bad_labels = ([t for t in dict_im_va.keys() if dict_im_va.get(t) == [-2.,-2.]])
len(image_name_bad_labels)

94060

Code to remove images with bad labels: 

In [27]:
len(dict_im_va)

414799

#### Defining function to get labels

In [28]:
def get_va_from_image(image_name, dict_im_va=dict_im_va):
    va = dict_im_va.get(image_name)
    if va==None:
        return(np.zeros(2))
    else:
        return va

## Reading images from folder and saving convolutional features (for now just training)

In [29]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [30]:
def get_data(path, target_size=(224,224), class_mode=None, shuffle=False, batch_size=10):
    gen = ImageDataGenerator()
    batches = gen.flow_from_directory(path, target_size=target_size, class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)
    return batches

In [31]:
batches_flow_from_dir = get_data(folder_train_data)

Found 419099 images belonging to 1349 classes.


## Defining custom flow_from_directory generator for labels

In [32]:
def flow_from_directory_va(flow_from_directory_gen):
    for x_batch in flow_from_directory_gen:
        idx = (flow_from_directory_gen.batch_index - 1) * flow_from_directory_gen.batch_size
        filenames_batch = flow_from_directory_gen.filenames[idx : idx + flow_from_directory_gen.batch_size]
        labels_batch = np.array([get_va_from_image(f) for f in filenames_batch])
        x_batch = preprocess_input(x_batch)
        yield x_batch, labels_batch

In [33]:
list_train_image_fnames = (glob.glob(folder_train_data+'/*/*'))
list_train_image_fnames

['/pylon5/ir5fpcp/amogh112/AffectNet/data_copy/Manually_Annotated_Images/392/ad7ed4b75c3cae6feedfee79a3e57b8545da805be42b06f03521be76.jpg',
 '/pylon5/ir5fpcp/amogh112/AffectNet/data_copy/Manually_Annotated_Images/392/d877da108d7659b732b49bf169d882ef2331ee679b3ac5a6635241e4.png',
 '/pylon5/ir5fpcp/amogh112/AffectNet/data_copy/Manually_Annotated_Images/392/4ef78580e7a098d68bb4b199b28451c68e7d269b72020adfce97f3fd.jpg',
 '/pylon5/ir5fpcp/amogh112/AffectNet/data_copy/Manually_Annotated_Images/392/319ab636ddfc28b298307513aca137743ae90dfa33fe235d9d53a566.jpg',
 '/pylon5/ir5fpcp/amogh112/AffectNet/data_copy/Manually_Annotated_Images/392/d07612a7ea425c33dcb85aeb53da73175529ffa872af31fab325c142.jpg',
 '/pylon5/ir5fpcp/amogh112/AffectNet/data_copy/Manually_Annotated_Images/392/074205c13b2a9e53ec2adbc49d420dc4f9c6525c20fdb54ef4699e05.jpg',
 '/pylon5/ir5fpcp/amogh112/AffectNet/data_copy/Manually_Annotated_Images/392/c4b074ea89d6f0484c50c2b605d41e3949965419726ab3c1b11e229c.jpg',
 '/pylon5/ir5fpcp/am

# Models - Defining, loading data, training, saving

### Model 1: VGG16 -> model_vgg_reg1

#### Defining the model

In [34]:
vgg_full = applications.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [35]:
vgg_full.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

Building the model on top of the conv part

In [36]:
p = 0.6

In [40]:
inp_top_model = Input(vgg_full.layers[-1].output_shape[1:])
# x = BatchNormalization(axis=1)(inp_top_model)
x = Dropout(p/4)(inp_top_model)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
x = Dropout(p)(x)
x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
x = Dropout(p/2)(x)
x_va = Dense(2, name='bva')(x)

In [41]:
top_model = Model(inp_top_model, x_va)

In [42]:
final_output = top_model(vgg_full.output)

In [43]:
model_vgg_reg1 = Model(vgg_full.input, final_output)

In [44]:
model_vgg_reg1.compile(loss='mean_squared_error',optimizer=optimizers.SGD(lr=0.001), metrics=['accuracy'])

In [45]:
model_vgg_reg1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

#### Loading batches, making destination folders

In [46]:
gen_va = flow_from_directory_va(batches_flow_from_dir)

#### Training of the model

In [47]:
model_vgg_reg1.fit_generator(gen_va, epochs=3, steps_per_epoch=10000)

Epoch 1/3


InternalError: Blas SGEMM launch failed : a.shape=(10, 512), b.shape=(10, 512), m=512, n=512, k=10
	 [[Node: training/SGD/gradients/model_2/dense_4/MatMul_grad/MatMul_1 = MatMul[T=DT_FLOAT, _class=["loc:@model_2/dense_4/MatMul"], transpose_a=true, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](model_2/dropout_5/cond/Merge, training/SGD/gradients/model_2/dense_4/Relu_grad/ReluGrad)]]

Caused by op 'training/SGD/gradients/model_2/dense_4/MatMul_grad/MatMul_1', defined at:
  File "/pylon5/ir5fpcp/amogh112/faus_dl/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/pylon5/ir5fpcp/amogh112/faus_dl/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/pylon5/ir5fpcp/amogh112/faus_dl/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/pylon5/ir5fpcp/amogh112/faus_dl/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in lau

In [137]:
new_model.summary(ImageDataGenerator())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1000)              134260544 
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2002      
Total params: 134,262,546
Trainable params: 134,262,546
Non-trainable params: 0
_________________________________________________________________


#### Unnecessary for now: saving conv features for VGG16

Saving conv features

In [40]:
layers = model_vgg16.layers
layer_idx = [index for index,layer in enumerate(layers) if type(layer) is Convolution2D][-1]
conv_layers, fc_layers = layers[:layer_idx+1], layers[layer_idx+1:]

In [41]:
model_vgg16_conv = Sequential(conv_layers)

In [42]:
model_vgg16_conv.layers

## Model 2: 

In [72]:
vgg_face_full = VGGFace(include_top=False, input_shape=(224, 224, 3), pooling='avg')

58916864/58909280 [==============================] - 22s 0us/step


In [73]:
vgg_face_full.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

### Model 2 - MobileNet

In [15]:
mobnet = applications.MobileNet()

17227776/17225924 [==============================] - 9s 1us/step


In [19]:
mobnet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 114, 114, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
__________